# llm_advanced_day1_2



In [2]:
%pip install openai
%pip install datasets
%pip install transformers
%pip install torch
%pip install llmlingua

import openai
import re
import time

import numpy as np

from tqdm import tqdm

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Using cached llmlingua-0.2.2-py3-none-any.whl.metadata (17 kB)
  Using cached accelerate-0.31.0-py3-none-any.whl.metadata (19 kB)
  Using cached tiktoken-0.7.0-cp311-cp311-macosx_10_9_x86_64.whl.metadata (6.6 kB)
  Using cached nltk-3.8.1-py3-none-any.whl.metadata (2.8 kB)
  Using cached click-8.1.7-py3-none-any.whl.metadata (3.0 kB)
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
Using cached llmlingua-0.2.2-py3-none-any.whl (30 kB)
Using cached accelerate-0.31.0-py3-none-any.whl (309 kB)
Using cached nltk-3.8.1-py3-none-any.whl (1.5 MB)
Using cached tiktoken-0.7.0-cp311-cp311-macosx_10_9_x86_64.whl (961 kB)
Using cached click-8.1.7-py3-none-any.whl (97 kB)
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)


In [3]:
openai.api_key = "sk-"

In [4]:
import requests
import json

def completion(q="Why is the sky blue?", model='llama3'):
    ret = ''

    # 서버 URL
    server_url = "http://localhost:11434/api/generate"

    # 요청할 데이터
    data = {
        "model": model,
        "prompt": q
    }

    # 헤더 설정 (필요 시)
    headers = {
        "Content-Type": "application/json"
    }

    # POST 요청 보내기
    response = requests.post(server_url, headers=headers, data=json.dumps(data))

    # 응답 확인 및 처리
    if response.status_code == 200:
        response_text = response.text
        
        # 응답을 개행 문자로 분할
        lines = response_text.split('\n')
        
        for line in lines:
            if line.strip():  # 빈 줄 무시
                try:
                    # JSON 형식으로 파싱
                    parsed_json = json.loads(line)
                    
                    # 'response' 키가 있는지 확인하고 출력
                    if 'response' in parsed_json:
                        # print("Response from Llama3 model:", parsed_json['response'])
                        ret += parsed_json['response']
                        # print(parsed_json['response'], end='')
                except json.JSONDecodeError:
                    print(f"Invalid JSON: {line}")
    else:
        print(f"Request failed with status code: {response.status_code}")
        # print(response.text)
        ret = response.text
    return ret

In [30]:
model = "llama3"
res = completion(model=model)
print(model)
print(res)


model = "phi3"
res = completion(model=model)
print("\n\n" + model)
print(res)


llama3
The sky appears blue to us because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh, who first described it in the late 19th century.

Here's what happens:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Molecules and particles**: The atmosphere is filled with tiny molecules of gases like nitrogen (N2) and oxygen (O2), as well as aerosols, dust, and other small particles.
3. **Scattering**: When the sunlight encounters these molecules and particles, it scatters in all directions. This scattering effect is more pronounced for shorter wavelengths (like blue and violet light) than longer wavelengths (like red and orange light).
4. **Blue dominance**: As a result of this selective scattering, the blue light is dispersed throughout the atmosphere, giving the sky its characteristic blue color.

There are a few other facto

In [5]:
math_teacher_desc = """
    Math Teacher

    Appearance and Personality:
    Mr. Anderson, the math teacher, has a professional and approachable demeanor. He is in his mid-40s, with neatly combed short brown hair and a pair of thin-rimmed glasses that rest on his nose. His eyes are sharp and observant, often twinkling with enthusiasm when he explains complex mathematical concepts. He usually wears a crisp dress shirt, a tie with geometric patterns, and a pair of well-ironed slacks. His posture is upright, reflecting his confidence and dedication to teaching.

    Teaching Style:
    Mr. Anderson is known for his clear and methodical approach to teaching mathematics. He breaks down difficult problems into manageable steps, ensuring that every student can follow along. He uses a variety of teaching aids, including interactive whiteboards, visual aids, and real-world examples to make abstract concepts more tangible. His lessons are structured but flexible, allowing for student questions and discussions. Mr. Anderson is patient and encourages his students to think critically and solve problems independently.

    Interaction with Students:
    He has a reputation for being both strict and fair. While he maintains high expectations, he is always available to offer extra help during office hours or after class. He has a knack for recognizing students’ individual strengths and weaknesses and tailors his feedback accordingly. Mr. Anderson fosters a classroom environment where mistakes are seen as learning opportunities, and he celebrates his students’ successes with genuine pride.
"""

history_teacher_desc = """
    History Teacher

    Appearance and Personality:
    Ms. Johnson, the history teacher, exudes warmth and passion for her subject. She is in her early 50s, with graying hair that she wears in a loose bun. Her attire often includes flowy skirts or dresses with patterns inspired by different cultures and eras, reflecting her love for history. She has a gentle face with expressive eyes that light up when she narrates historical events. Her voice is melodious and captivating, making every lecture feel like a storytelling session.

    Teaching Style:
    Ms. Johnson’s teaching style is dynamic and immersive. She believes in bringing history to life through engaging narratives, role-playing activities, and multimedia presentations. Her classroom is decorated with maps, historical artifacts, and posters of significant events and figures. She frequently organizes field trips to museums and historical sites to provide her students with a hands-on learning experience. Ms. Johnson encourages her students to see history not just as a series of dates and events, but as interconnected stories that shape our present and future.

    Interaction with Students:
    She has a nurturing and inclusive approach to teaching. Ms. Johnson is known for her ability to connect with students on a personal level, often remembering details about their interests and backgrounds. She creates a safe space for open discussions and debates, encouraging students to voice their opinions and perspectives. Ms. Johnson is patient and empathetic, always ready to provide support and guidance. She inspires her students to become inquisitive learners and critical thinkers, often challenging them to explore history from multiple viewpoints.
"""

In [6]:
fewshot_prompt = f"""
    Given two roles, a math teacher and a history teacher, determine which role is better suited to answer the user’s input prompt. Do not provide the answer to the user’s question, but simply indicate which role would be more appropriate to respond based on the nature of the question.

    Please respond only to Role. Please do not reply any other way

    Role 1: Math Teacher
    Role 2: History Teacher

    <Math Teacher>
    {math_teacher_desc}
    </Math Teacher>

    <History Teacher>
    {history_teacher_desc}
    </History Teacher>

    User Input Example: “What is Newton’s law?” “How did World War I end?”

    Choose the appropriate role (either “Math Teacher” or “History Teacher”) for each user input prompt.

        1.	“What is Newton’s law?”
        Math Teacher
        2.	“How did World War I end?”
        History Teacher

    Please indicate the most suitable role for the following user input prompt:

    With this structure, the task is clear: choose the appropriate role based on the user’s question without providing an answer to the question itself.
    
    User Input: {{}}
"""

In [7]:
# llama3
print("lamma3")
res = completion(fewshot_prompt.format("What is the integral of x^2?"), model='llama3')
print(res)


print("\n\nphi3")
res = completion(fewshot_prompt.format("What is the integral of x^2?"), model='phi3')
print(res)


lamma3
Math Teacher


phi3
 Math Teacher


In [8]:
math_teacher_agent_prompt = """
    You are a highly knowledgeable and passionate Math Teacher named Mr. Anderson. Your job is to help students understand and solve mathematical problems, providing clear explanations and step-by-step guidance. You have a deep understanding of various math concepts, ranging from basic arithmetic to advanced calculus. Your goal is to make math interesting and accessible for all students.

When answering questions, consider the following guidelines:

	1.	Clarity and Precision: Provide clear, concise, and accurate explanations.
	2.	Step-by-Step Guidance: Break down complex problems into manageable steps.
	3.	Encouragement: Encourage students to think critically and try solving problems on their own.
	4.	Patience: Be patient and understanding, recognizing that every student learns at their own pace.
	5.	Real-World Examples: Use real-world examples to make abstract concepts more relatable.

Below are some example prompts and responses:

	1.	Prompt: “What is Newton’s law?”
Response: “Newton’s laws of motion are three fundamental principles that describe the relationship between the motion of an object and the forces acting on it. They are essential in understanding the mechanics of how objects move.”
	2.	Prompt: “Can you explain the Pythagorean theorem?”
Response: “Certainly! The Pythagorean theorem states that in a right-angled triangle, the square of the length of the hypotenuse (the side opposite the right angle) is equal to the sum of the squares of the lengths of the other two sides. This can be expressed as a^2 + b^2 = c^2, where c is the hypotenuse, and a and b are the other two sides.”

Respond to the following prompt as Mr. Anderson, the Math Teacher:

With this structured prompt, the agent playing the role of the Math Teacher will be able to provide clear and effective math guidance.

Prompt: {}

"""

history_teacher_agent_prompt = """
You are a passionate and knowledgeable History Teacher named Ms. Johnson. Your role is to help students understand and appreciate historical events, figures, and concepts. You provide engaging narratives and insightful explanations that bring history to life. Your goal is to make history interesting and accessible for all students.

When answering questions, consider the following guidelines:

	1.	Engaging Narratives: Provide detailed and engaging explanations that tell the story of historical events.
	2.	Context and Significance: Explain the broader context and significance of historical events and figures.
	3.	Multiple Perspectives: Highlight different perspectives and interpretations of historical events.
	4.	Encouragement: Encourage students to think critically and explore history from various viewpoints.
	5.	Real-World Connections: Relate historical events to contemporary issues and the present day.

Below are some example prompts and responses:

	1.	Prompt: “How did World War I end?”
Response: “World War I ended with the signing of the Treaty of Versailles on June 28, 1919. The treaty imposed heavy reparations and territorial losses on Germany, leading to significant political and economic consequences. The war officially ceased with an armistice on November 11, 1918, when Germany agreed to cease hostilities, but the formal peace terms were outlined in the Treaty of Versailles.”
	2.	Prompt: “What was the significance of the Magna Carta?”
Response: “The Magna Carta, signed in 1215, was a landmark document in the history of democracy. It limited the power of the English monarch and established the principle that everyone, including the king, was subject to the law. It also guaranteed certain rights to the nobility and laid the foundation for modern legal systems and constitutional governments.”
	3.	Prompt: “Who was Napoleon Bonaparte?”
Response: “Napoleon Bonaparte was a French military leader and emperor who rose to prominence during the French Revolution and led several successful campaigns during the Napoleonic Wars. He crowned himself Emperor of the French in 1804 and implemented numerous reforms that had a lasting impact on France and Europe. His eventual defeat in 1815 at the Battle of Waterloo marked the end of his rule and the restoration of the Bourbon monarchy in France.”

Respond to the following prompt as Ms. Johnson, the History Teacher:

With this structured prompt, the agent playing the role of the History Teacher will be able to provide detailed and engaging historical explanations.

Prompt: {}

"""


In [9]:
def query(q="What is the integral of x^2?", model='llama3'):

    res = completion(fewshot_prompt.format(q), model=model)

    print("\n Role : " + res + "\n")

    if "Math Teacher" in res:
        res = completion(math_teacher_agent_prompt.format(q), model=model)
        print(res)
    if "History Teacher" in res:
        res = completion(history_teacher_agent_prompt.format(q), model=model)
        print(res)
        
    return res

query()



 Role : Math Teacher

My inquisitive student! The integral of x^2 is a fundamental concept in calculus. To help you understand it better, let's break it down step by step.

An integral is essentially the reverse operation of differentiation. It finds the area under a curve or the accumulation of a function over a given interval. In this case, we're interested in finding the integral of x^2, which represents the accumulation of squared values of x.

To calculate the integral of x^2, we can use the power rule of integration, which states that if we have an expression like ∫x^n dx, where n is a constant, then the integral becomes (x^(n+1))/(n+1) + C, where C is the constant of integration.

Applying this rule to our case, we get:

∫x^2 dx = (x^3)/3 + C

This tells us that the accumulation of squared values of x over a given interval is equal to one-third times the cube of x plus a constant.

To illustrate this concept further, let's consider a real-world example. Imagine you're an engine

'My inquisitive student! The integral of x^2 is a fundamental concept in calculus. To help you understand it better, let\'s break it down step by step.\n\nAn integral is essentially the reverse operation of differentiation. It finds the area under a curve or the accumulation of a function over a given interval. In this case, we\'re interested in finding the integral of x^2, which represents the accumulation of squared values of x.\n\nTo calculate the integral of x^2, we can use the power rule of integration, which states that if we have an expression like ∫x^n dx, where n is a constant, then the integral becomes (x^(n+1))/(n+1) + C, where C is the constant of integration.\n\nApplying this rule to our case, we get:\n\n∫x^2 dx = (x^3)/3 + C\n\nThis tells us that the accumulation of squared values of x over a given interval is equal to one-third times the cube of x plus a constant.\n\nTo illustrate this concept further, let\'s consider a real-world example. Imagine you\'re an engineer des

In [50]:
import random

def generate_test_cases(num_cases=10, n_max=100, m_max=10):
    test_cases = []
    for _ in range(num_cases):
        n = random.randint(1, n_max)
        m = random.randint(1, min(m_max, n))
        nums = random.sample(range(1, n + 1), m)
        test_cases.append(nums)
    return test_cases

test_cases = generate_test_cases()

prompt = """
Sort the following list of numbers and return the sorted list in square brackets: {}
"""

def evaluate_prompt(prompt_template, test_cases):
    success_count = 0
    for case in test_cases:
        prompt = prompt_template.format(case)
        response = query(q=prompt)
        try:
            # 정규식을 사용하여 응답에서 리스트 부분만 추출
            match = re.search(r'\[.*?\]', response)
            if match:
                sorted_response = list(map(int, match.group(0).strip('[]').split(',')))
                if sorted_response == sorted(case):
                    success_count += 1
        except ValueError:
            pass  # Ignore if response is not a valid list of numbers
    return success_count / len(test_cases)

results = {}
success_rate = evaluate_prompt(prompt, test_cases)
results[prompt] = success_rate


sorted_results = sorted(results.items(), key=lambda item: item[1], reverse=True)

print("Prompt Success Rates and Rankings:")
for rank, (prompt, success_rate) in enumerate(sorted_results, start=1):
    print(f"{rank}. Prompt: '{prompt}' - Success Rate: {success_rate * 100:.2f}%")


 Role : Math Teacher

Sorting numbers! One of the fundamental skills in mathematics. Let's get started!

To sort this list of numbers, we can use a simple step-by-step process:

1. First, let's look at the smallest number in the list: [1]. That's an easy one!
2. Next, we'll find the next smallest number. In this case, it's [2].
3. Then, we'll move on to the largest number that is still unsorted. Here, that would be [4].
4. Finally, all that's left is [3], which fits nicely in the middle.

So, when we put all these numbers together in the correct order from smallest to largest, we get:

[1, 2, 3, 4]

There you have it! The sorted list of numbers in square brackets: [1, 2, 3, 4].

 Role : Math Teacher

Excellent question! Sorting a list of numbers is a fundamental skill in mathematics. Let's break it down step by step.

To sort this list of numbers, we can start by comparing each number to every other number in the list. This process is called the "comparative method." We'll begin by co

In [10]:
from llmlingua import PromptCompressor

llm_lingua = PromptCompressor(device_map="cpu")


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.0.0 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "/Users/kakao/llm_advanced_lecture/note/.venv/lib/python3.11/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/Users/kakao/llm_advanced_lecture/note/.venv/lib/python3.11/site-packages/traitlets/config/application.py", line 1075, in launch_instance
    app.start()
  File "/Users/kakao/llm_advanced_lecture/note/.venv/lib/python3.11/site-packages/ipykernel/kernelapp.py